Continuous H-MCTS Multi Plan (w/ wandb)
================================
Makes multiple plan in a hierarchy and move on to lower level

*without extendable node

### 1. Login to wandb

In [ ]:
import wandb
wandb.login()

### 2. Import packages

In [ ]:
import math
import json
from itertools import product
import os
import sys

In [ ]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils import *
sys.path.append(ROOT_DIR)

from src.Env.Grid.Cont_Grid import Continuous_Grid
from src.Env.Grid.Higher_Grids_HW import HighLevelGrids2
from src.Planners.H_MCTS_continuous.version.H_MCTS_Cont_multi import H_MCTS_Cont

In [ ]:
# Basic Setup for environment
l1_rows = 8
l1_cols = 8
l1_width = 2
l1_height = 2
goal_distance = 3
grid_setting = (l1_rows, l1_cols, l1_width, l1_height, goal_distance)
num_barrier = 15

In [ ]:
# Test code
# Input: 
#   param
#       0. explorationConstant
#       1. iter_Limit
#       2. alpha (PW)
#       3. constant_c (PW)
#       4. gamma
# (NEW) 5. reward_function_weight
#       6. l1_goal_reward
#       7. l1_subgoal_reward
#   grid_setting
#   folder_name: for local save

def test_h_mcts_multi(param, grid_setting, folder_name):
    mcts_result = {}
    mcts_result["iter_cnt"] = {}
    mcts_success_rate = 0

    explorationConstant = param[0]
    iter_Limit = param[1]
    alpha = param[2]
    constant_c = param[3]
    gamma = param[4]
    reward_function_weight = param[5]
    l1_goal_reward=param[6]
    l1_subgoal_reward=param[7]

    new_grid_setting = list(grid_setting)
    new_grid_setting.append(reward_function_weight)
    grid_setting = tuple(new_grid_setting)

    for random_seed in range(100):
        print("Random_seed", random_seed)
        MCTS = H_MCTS_Cont(grid_setting, random_seed=random_seed,
                            H_level=2,
                            num_barrier=15,
                            explorationConstant_h=1 / math.sqrt(explorationConstant),
                            explorationConstant_l=1 / math.sqrt(explorationConstant),
                            alpha=alpha,
                            constant_c=constant_c,
                            gamma=gamma,
                            iter_Limit = iter_Limit,
                            l1_goal_reward=l1_goal_reward,
                            l1_subgoal_reward=l1_subgoal_reward)
        traj, success, iter = MCTS.search(
                        tree_save_path="{}/tree/{}.png".format(folder_name, random_seed),
                        traj_save_path="{}/found_path/{}.txt".format(folder_name, random_seed))
        iter += 1
        mcts_result["iter_cnt"][random_seed] = iter

        if success:
            mcts_success_rate += 1
            MCTS.cont_env.plot_grid(0, traj, "{}/traj/{}.png".format(folder_name, random_seed))
            print(f'success with {iter}')
        else:
            MCTS.cont_env.plot_grid(0, traj, "{}/traj/{}.png".format(folder_name, random_seed))
            print('Failed')
        
        wandb.log({f"{random_seed}": wandb.Image("{}/tree/{}.png".format(folder_name, random_seed))})
        wandb.log({f"{random_seed}": wandb.Image("{}/traj/{}.png".format(folder_name, random_seed))})

    mcts_result["success_rate"] = mcts_success_rate
    with open("{}/result.json".format(folder_name), 'w') as result_file:
        json.dump(mcts_result, result_file)
    x_values, y_values = cumul_plot(iter_Limit, mcts_result, folder_name)
    wandb.log({"iteration_plot": wandb.Image("{}/success_rate.png".format(folder_name))})
    
    data = [[x, y] for (x, y) in zip(x_values, y_values)]
    table = wandb.Table(data=data, columns = ["x", "y"])
    wandb.log(
        {"Iteration_Plot" : wandb.plot.line(table, "x", "y",
            title="Iteration vs Success Rate Plot")})
    return mcts_result


In [ ]:
# Launch experiment name
total_runs = 1
for run in range(total_runs):
    # 1. Start a new run to track script
    explorationConstant = [0.25, 0.5, 1.0, 1.5]
    iter_Limit = [10000]
    alpha = [0.01, 0.025, 0.05]
    constant_c = [2]
    gamma = [1]
    reward_function_weight = [0.2]
    l1_goal_reward = [8]
    l1_subgoal_reward = [4]
    grid_setting = grid_setting

    # Create a list of parameter arrays
    parameters = [explorationConstant, iter_Limit, alpha, constant_c, gamma, reward_function_weight, l1_goal_reward, l1_subgoal_reward]

    # Generate all possible combinations
    param_combinations = list(product(*parameters))

    # Print the combinations
    for param in param_combinations:
        folder_name = os.path.join(H_MCTS_MULTI_EXPERIMENT_DIR, "multi-plan_h_mcts_{}_{}_{}_{}_{}_{}_g{}_sg_{}" \
                    .format(param[0], param[1], param[2], param[3], param[4], param[5], param[6], param[7]))
        folder_exists = make_param_dir(folder_name)
        print("Param to check", param)


        if not folder_exists:

            wandb.init(
                # Project name
                project='h-mcts-multi',
                # Run name
                name=f"multi_plan_experiment_{run}_{param[0]}_{param[1]}_{param[2]}_{param[3]}_{param[4]}_{param[5]}_g{param[6]}_sg{param[7]}",
                # Track hyperparameters and run metadata
                config={
                    "goal_distance": goal_distance,
                    "num_barrier": num_barrier,
                    "H_level": 2,
                    "explorationConstant": param[0],
                    "iter_Limit": param[1],
                    "alpha": param[2],
                    "constant_c": param[3],
                    "gamma": param[4],
                    "reward_function_weight": param[5],
                    "l1_goal_reward": param[6],
                    "l1_subgoal_reward": param[7],
                }
            )
            wandb.run.log_code(".")
            
            make_param_dir("{}/traj".format(folder_name))
            make_param_dir("{}/tree".format(folder_name))
            make_param_dir("{}/found_path".format(folder_name))
            mcts_result = test_h_mcts_multi(param, grid_setting, folder_name)
            wandb.log(mcts_result)
    
        # This simple block simulates a training loop loggin metrics
        wandb.finish()